In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

2023-05-24 09:21:15.657276: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(['EIN', 'NAME'], axis=1)

In [3]:
# Determine the number of unique values in each column.
unique_counts = application_df.nunique()
unique_counts

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [4]:
# Look at APPLICATION_TYPE value counts for binning
application_type_counts = application_df['APPLICATION_TYPE'].value_counts()
application_type_counts

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [5]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_types_to_replace = list(application_type_counts[application_type_counts < 500].index)

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [6]:
# Look at CLASSIFICATION value counts for binning
classification_counts = application_df['CLASSIFICATION'].value_counts()

In [7]:
# You may find it helpful to look at CLASSIFICATION value counts >1
classification_counts

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [8]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classifications_to_replace = list(classification_counts[classification_counts < 1000].index)

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [9]:
# Convert categorical data to numeric with `pd.get_dummies`
numeric_df = pd.get_dummies(application_df)

In [10]:
# Split our preprocessed data into our features and target arrays
X = numeric_df.drop('IS_SUCCESSFUL', axis=1).values
y = numeric_df['IS_SUCCESSFUL'].values
# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [11]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [12]:
# Optimized Model
optimized_model = tf.keras.models.Sequential()

In [13]:
# Add layers
optimized_model.add(tf.keras.layers.Dense(units=120, activation="relu", input_dim=X.shape[1]))
optimized_model.add(tf.keras.layers.Dense(units=30, activation="relu"))
optimized_model.add(tf.keras.layers.Dense(units=15, activation="relu"))
optimized_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

optimized_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 120)               5280      
                                                                 
 dense_1 (Dense)             (None, 30)                3630      
                                                                 
 dense_2 (Dense)             (None, 15)                465       
                                                                 
 dense_3 (Dense)             (None, 1)                 16        
                                                                 
Total params: 9,391
Trainable params: 9,391
Non-trainable params: 0
_________________________________________________________________


In [14]:
# Compile model
optimized_model.compile(loss="binary_crossentropy", optimizer="adam",  metrics=['accuracy'])

In [15]:
# Train model
optimized_model.fit(X_train_scaled, y_train, epochs=150, shuffle=True, verbose=2)

Epoch 1/150
804/804 - 1s - loss: 0.5718 - accuracy: 0.7201 - 1s/epoch - 2ms/step
Epoch 2/150
804/804 - 1s - loss: 0.5553 - accuracy: 0.7302 - 622ms/epoch - 774us/step
Epoch 3/150
804/804 - 1s - loss: 0.5525 - accuracy: 0.7315 - 643ms/epoch - 800us/step
Epoch 4/150
804/804 - 1s - loss: 0.5505 - accuracy: 0.7320 - 632ms/epoch - 787us/step
Epoch 5/150
804/804 - 1s - loss: 0.5495 - accuracy: 0.7325 - 634ms/epoch - 788us/step
Epoch 6/150
804/804 - 1s - loss: 0.5489 - accuracy: 0.7333 - 619ms/epoch - 770us/step
Epoch 7/150
804/804 - 1s - loss: 0.5481 - accuracy: 0.7353 - 632ms/epoch - 786us/step
Epoch 8/150
804/804 - 1s - loss: 0.5476 - accuracy: 0.7340 - 633ms/epoch - 787us/step
Epoch 9/150
804/804 - 1s - loss: 0.5464 - accuracy: 0.7363 - 610ms/epoch - 758us/step
Epoch 10/150
804/804 - 1s - loss: 0.5462 - accuracy: 0.7351 - 630ms/epoch - 783us/step
Epoch 11/150
804/804 - 1s - loss: 0.5460 - accuracy: 0.7354 - 661ms/epoch - 823us/step
Epoch 12/150
804/804 - 1s - loss: 0.5455 - accuracy: 0.73

Epoch 96/150
804/804 - 1s - loss: 0.5356 - accuracy: 0.7410 - 598ms/epoch - 744us/step
Epoch 97/150
804/804 - 1s - loss: 0.5349 - accuracy: 0.7406 - 638ms/epoch - 793us/step
Epoch 98/150
804/804 - 1s - loss: 0.5363 - accuracy: 0.7414 - 733ms/epoch - 912us/step
Epoch 99/150
804/804 - 1s - loss: 0.5357 - accuracy: 0.7416 - 675ms/epoch - 840us/step
Epoch 100/150
804/804 - 1s - loss: 0.5349 - accuracy: 0.7416 - 587ms/epoch - 730us/step
Epoch 101/150
804/804 - 1s - loss: 0.5349 - accuracy: 0.7416 - 605ms/epoch - 753us/step
Epoch 102/150
804/804 - 1s - loss: 0.5348 - accuracy: 0.7413 - 587ms/epoch - 730us/step
Epoch 103/150
804/804 - 1s - loss: 0.5348 - accuracy: 0.7414 - 591ms/epoch - 735us/step
Epoch 104/150
804/804 - 1s - loss: 0.5351 - accuracy: 0.7415 - 588ms/epoch - 732us/step
Epoch 105/150
804/804 - 1s - loss: 0.5353 - accuracy: 0.7414 - 595ms/epoch - 740us/step
Epoch 106/150
804/804 - 1s - loss: 0.5349 - accuracy: 0.7412 - 588ms/epoch - 732us/step
Epoch 107/150
804/804 - 1s - loss: 0

In [16]:
# Evaluate the model using the test data
model_loss, model_accuracy = optimized_model.evaluate(X_test_scaled, y_test, verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5607 - accuracy: 0.7252 - 283ms/epoch - 1ms/step
Loss: 0.5606889724731445, Accuracy: 0.7252478003501892


In [17]:
# Export optimized model to HDF5 file
optimized_model.save("AlphabetSoupCharity_Optimization.h5")